For later: 
- this is no account for trades made between seasons or during seasons
- normalizing the pbp arrays made the difference between always predicting wins and actually having variation


great article for Paramater tuning for SCV: https://medium.com/all-things-ai/in-depth-parameter-tuning-for-svc-758215394769

In [1]:
import datetime
import string
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from basketball_reference_scraper.pbp import get_pbp
from basketball_reference_scraper.constants import TEAM_TO_TEAM_ABBR
from basketball_reference_scraper.seasons import get_schedule

In [2]:
teams = [
 'PHILADELPHIA 76ERS',
 'ATLANTA HAWKS',
 'BOSTON CELTICS',
 'BROOKLYN NETS',
 'CHICAGO BULLS',
 'CHARLOTTE HORNETS',
 'CLEVELAND CAVALIERS',
 'DALLAS MAVERICKS',
 'DENVER NUGGETS',
 'DETROIT PISTONS',
 'GOLDEN STATE WARRIORS',
 'HOUSTON ROCKETS',
 'INDIANA PACERS',
 'LOS ANGELES CLIPPERS',
 'LOS ANGELES LAKERS',
 'MEMPHIS GRIZZLIES',
 'MIAMI HEAT',
 'MILWAUKEE BUCKS',
 'MINNESOTA TIMBERWOLVES',
 'NEW ORLEANS PELICANS',
 'NEW YORK KNICKS',
 'OKLAHOMA CITY THUNDER',
 'ORLANDO MAGIC',
 'PHOENIX SUNS',
 'PORTLAND TRAIL BLAZERS',
 'SACRAMENTO KINGS',
 'SAN ANTONIO SPURS',
 'TORONTO RAPTORS',
 'UTAH JAZZ',
 'WASHINGTON WIZARDS'
]
 
philly =  'PHILADELPHIA 76ERS'  # philly is separated because python doesn't like digits in the name!

In [3]:
#  Indexing pbp dataframe
QTR_IDX = 0
TIME_IDX = 1
HOME_IDX = 2
AWAY_IDX = 3

#  Indexing numpy array "plays"
TEAM_IDX = 0
TIME_IDX = 1
SCOREPLAY_IDX = 2
BENEFIT_IDX = 3
DETRIMENT_IDX = 4
MOMENTUM_IDX = 5

results = ['LOSS','WIN']

In [4]:
def shuffle_arrays(a, b, c=None):
    assert len(a) == len(b) and len(b) == len(c)
    p = np.random.permutation(len(a))
    a = [a[i] for i in p]
    b = [b[i] for i in p]
    c = [c[i] for i in p]
    
    return a, b, c

def name_to_abbr(team):
    return TEAM_TO_TEAM_ABBR[team.upper()]

def calculate_elapsed_time(remaining, quarter):
    qtr = '12:00.0' # 12-minute regulation quarters
    ot = '5:00.0'  # 5-minute overtime periods
    format = '%M:%S.%f'

    if 'OT' not in str(quarter): #  play occurs during regulation
        period = qtr
        quarter_progression = datetime.timedelta(minutes=(quarter-1)*12) 
    else: #  play occurs during overtime
        period = ot
        ot_period = int(quarter.strip(string.ascii_letters))
        quarter_progression = datetime.timedelta(minutes=(ot_period-1)*5)  
        
    return (quarter_progression + 
            datetime.datetime.strptime(period, format) - 
            datetime.datetime.strptime(remaining, format))

# def penalty_play(play):
#     results = ['foul']

def scoring_play(play):
    results = ['makes 2-pt', 'makes 3-pt']
    if any(word in play for word in results):
        return 1
    return 0
    
def benifit_play(play):
    results = ['rebound', 'foul']
    if any(word in play for word in results):
        return 1
    return 0

def detriment_play(play):
    results = ['turnover', 'miss', 'timeout']
    if any(word in play for word in results):
        return 1
    return 0
    
def process_play(time, play):
    return np.array([time, scoring_play(play), benifit_play(play), detriment_play(play)])

def calculate_momentum(play_arr):
    return (1+play_arr[SCOREPLAY_IDX]) * (1+play_arr[BENEFIT_IDX]) - 2*play_arr[DETRIMENT_IDX]
    
def momentum_checks(plays):
    playcount = plays.shape[0]
    
    for play in range(playcount):
        idx = play  # start from the current play
        while idx >= 0:
            idx -= 1
            if plays[play][TIME_IDX] - plays[idx][TIME_IDX] > 60:
                break
            plays[play][MOMENTUM_IDX] += calculate_momentum(plays[idx])

    return plays

In [5]:
#  Get the nba schedule (all games)
year = 2021
nba = get_schedule(year, playoffs=False)

In [ ]:
def run(nba, model=LinearSVC(random_state=0), printouts=True):

    accuracies = []
    predicted_team_records = {team: 0 for team in teams}

    for _team in teams:
        team = _team.title() if _team != philly else 'Philadelphia 76ers'
        predicted_team_records[team] = predicted_team_records.pop(_team)
        
        #  Remove games that have not yet been played
        season = nba[~np.isnan(nba.VISITOR_PTS)]
        season = nba[~np.isnan(nba.HOME_PTS)]

        #  Select games played by a specific team if desired
        team_season = pd.concat((season[season.VISITOR == team], season[season.HOME== team]), axis=0)


        #  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

        games_pbp = []
        wins = []
        game_summaries = []

        playcount_stopper = 200

        #  Extract parameters used by play-by-play functions 
        for idx,row in team_season.iterrows():
            date = row['DATE']
            away = name_to_abbr(row['VISITOR'])
            home = name_to_abbr(row['HOME'])
            home_winner = row['HOME_PTS'] > row['VISITOR_PTS']
            winner = home if home_winner else away

            summary = f'{date.date()}: {away} @ {home} | winner: {winner}'
            game_summaries.append(summary)

            wins.append(1 if winner == name_to_abbr(team) else 0)

            #  Begin processing play-by-play data on a per-game basis
            pbp = get_pbp(date, home, away)
            pbp_playcount = len(pbp)

            # Stop collecting pbp data after n plays 
            # Comment this line to use the full pbp data; 
            # this may introduce problems as teams play different number of games (reschedule, cancel, playoffs...)
            pbp_playcount = playcount_stopper


            plays = np.zeros((pbp_playcount, 6))  # [HOME?, time (secs), score, benefit, detriment, momentum_score]    

            for play in range(pbp_playcount):
                elapsed_gametime = calculate_elapsed_time(pbp.iloc[play][TIME_IDX], pbp.iloc[play][QTR_IDX])
                action_home = pbp.iloc[play][HOME_IDX]
                action_away = pbp.iloc[play][AWAY_IDX]

                if action_home is not np.nan:
                    plays[play] = np.hstack((1, process_play(elapsed_gametime.seconds, action_home.lower()), 0))
                else:
                    plays[play] = np.hstack((0, process_play(elapsed_gametime.seconds, action_away.lower()), 0))

            plays = momentum_checks(plays)
            plays = plays / plays.max(axis=0)

            games_pbp.append(plays)

        #  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

        if printouts:
                print(np.array(games_pbp).shape)
        game_count = len(games_pbp)

        games_pbp, wins, game_summaries = shuffle_arrays(np.array(games_pbp), np.array(wins), game_summaries)

        tts_idx = round(game_count*0.75)  # train-test-split index

        X_train = np.array(games_pbp)[:tts_idx].reshape(tts_idx,-1)
        y_train = np.array(wins)[:tts_idx].reshape(-1,)

        X_test = np.array(games_pbp)[tts_idx:].reshape(game_count-tts_idx,-1)
        y_test = np.array(wins)[tts_idx:].reshape(-1,)

        #  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

        clf = model.fit(X_train, y_train)

        preds = clf.predict(X_test)
        acc = accuracy_score(y_test, preds)
        accuracies.append(acc)

        if printouts:
                print(f'Team: {team} accuracy = {100*acc}%')

        test_predictions = [results[i] for i in preds]

        for i in range(tts_idx, game_count): # indexing only the test data
            if preds[i-tts_idx] == 1:
                predicted_team_records[team] += 1
            if printouts:
                print(f'{game_summaries[i]} ||| Prediction: {name_to_abbr(team)} {test_predictions[i-tts_idx]}')

        print('\n')
        #  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
        
    test_games = game_count-tts_idx  # The number of games that were used for testing
    
    print(f'League accuracy average: {np.mean(accuracies)*100:.2f}%')
    print(f'League wins average: {np.mean(list(predicted_team_records.values()))/test_games:.3f}%')

    for team in range(len(teams)):
        _team = teams[team].title() if teams[team] != philly else 'Philadelphia 76ers'
        print(f'Win rate: {predicted_team_records[_team]/test_games:.3f} | Correctly predicted {accuracies[team]*100:.2f}% for team {teams[team]}')

In [7]:
# Vanilla LinearSVC
run(nba)

(84, 200, 6)
Team: Philadelphia 76ers accuracy = 57.14285714285714%
2021-03-25: PHI @ LAL | winner: PHI ||| Prediction: PHI WIN
2021-06-08: ATL @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-02-17: HOU @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-03-23: PHI @ GSW | winner: PHI ||| Prediction: PHI WIN
2021-01-06: WAS @ PHI | winner: PHI ||| Prediction: PHI LOSS
2021-05-23: WAS @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-05-29: PHI @ WAS | winner: PHI ||| Prediction: PHI WIN
2021-05-26: WAS @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-04-03: MIN @ PHI | winner: PHI ||| Prediction: PHI LOSS
2021-05-02: PHI @ SAS | winner: PHI ||| Prediction: PHI WIN
2021-01-25: PHI @ DET | winner: DET ||| Prediction: PHI WIN
2021-03-30: PHI @ DEN | winner: DEN ||| Prediction: PHI WIN
2021-05-07: NOP @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-02-27: CLE @ PHI | winner: CLE ||| Prediction: PHI WIN
2021-01-14: MIA @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-05-11: PHI @ IND | winner

(79, 200, 6)
Team: Dallas Mavericks accuracy = 70.0%
2021-04-12: PHI @ DAL | winner: PHI ||| Prediction: DAL WIN
2021-01-04: DAL @ HOU | winner: DAL ||| Prediction: DAL WIN
2021-01-01: MIA @ DAL | winner: DAL ||| Prediction: DAL WIN
2021-02-08: MIN @ DAL | winner: DAL ||| Prediction: DAL WIN
2021-01-30: PHO @ DAL | winner: PHO ||| Prediction: DAL WIN
2021-02-04: GSW @ DAL | winner: GSW ||| Prediction: DAL LOSS
2021-01-27: DAL @ UTA | winner: UTA ||| Prediction: DAL LOSS
2021-03-31: DAL @ BOS | winner: DAL ||| Prediction: DAL LOSS
2021-05-04: DAL @ MIA | winner: DAL ||| Prediction: DAL WIN
2021-05-25: DAL @ LAC | winner: DAL ||| Prediction: DAL WIN
2021-02-14: POR @ DAL | winner: POR ||| Prediction: DAL LOSS
2021-05-09: DAL @ CLE | winner: DAL ||| Prediction: DAL WIN
2021-03-17: LAC @ DAL | winner: DAL ||| Prediction: DAL WIN
2020-12-27: DAL @ LAC | winner: DAL ||| Prediction: DAL WIN
2021-03-11: DAL @ OKC | winner: OKC ||| Prediction: DAL WIN
2021-01-29: DAL @ UTA | winner: UTA ||| Pre

(79, 200, 6)
Team: Los Angeles Lakers accuracy = 40.0%
2020-12-22: LAC @ LAL | winner: LAC ||| Prediction: LAL WIN
2021-04-10: LAL @ BRK | winner: LAL ||| Prediction: LAL WIN
2021-03-16: MIN @ LAL | winner: LAL ||| Prediction: LAL LOSS
2021-04-22: LAL @ DAL | winner: DAL ||| Prediction: LAL LOSS
2021-01-28: LAL @ DET | winner: DET ||| Prediction: LAL WIN
2021-01-03: LAL @ MEM | winner: LAL ||| Prediction: LAL LOSS
2021-03-21: LAL @ PHO | winner: PHO ||| Prediction: LAL LOSS
2021-06-03: PHO @ LAL | winner: PHO ||| Prediction: LAL WIN
2021-02-14: LAL @ DEN | winner: DEN ||| Prediction: LAL WIN
2021-03-26: CLE @ LAL | winner: LAL ||| Prediction: LAL WIN
2021-02-28: GSW @ LAL | winner: LAL ||| Prediction: LAL WIN
2021-01-10: LAL @ HOU | winner: LAL ||| Prediction: LAL LOSS
2021-01-08: CHI @ LAL | winner: LAL ||| Prediction: LAL LOSS
2021-02-16: LAL @ MIN | winner: LAL ||| Prediction: LAL LOSS
2021-02-08: OKC @ LAL | winner: LAL ||| Prediction: LAL WIN
2021-05-07: LAL @ POR | winner: POR ||

(72, 200, 6)
Team: Oklahoma City Thunder accuracy = 61.111111111111114%
2021-05-01: IND @ OKC | winner: IND ||| Prediction: OKC LOSS
2021-03-18: OKC @ ATL | winner: ATL ||| Prediction: OKC WIN
2021-02-17: OKC @ MEM | winner: MEM ||| Prediction: OKC LOSS
2021-05-14: UTA @ OKC | winner: UTA ||| Prediction: OKC LOSS
2021-01-24: OKC @ LAC | winner: LAC ||| Prediction: OKC WIN
2021-04-21: OKC @ IND | winner: IND ||| Prediction: OKC LOSS
2021-04-14: GSW @ OKC | winner: GSW ||| Prediction: OKC LOSS
2021-03-13: NYK @ OKC | winner: NYK ||| Prediction: OKC LOSS
2021-03-04: OKC @ SAS | winner: OKC ||| Prediction: OKC LOSS
2021-04-18: OKC @ TOR | winner: TOR ||| Prediction: OKC LOSS
2021-04-03: OKC @ POR | winner: POR ||| Prediction: OKC WIN
2020-12-31: NOP @ OKC | winner: NOP ||| Prediction: OKC LOSS
2021-03-16: OKC @ CHI | winner: CHI ||| Prediction: OKC LOSS
2021-04-10: PHI @ OKC | winner: PHI ||| Prediction: OKC LOSS
2021-01-02: OKC @ ORL | winner: OKC ||| Prediction: OKC LOSS
2021-04-13: OKC 

(83, 200, 6)
Team: Utah Jazz accuracy = 42.857142857142854%
2021-03-22: UTA @ CHI | winner: UTA ||| Prediction: UTA WIN
2020-12-26: MIN @ UTA | winner: MIN ||| Prediction: UTA WIN
2021-05-10: UTA @ GSW | winner: GSW ||| Prediction: UTA WIN
2021-05-16: UTA @ SAC | winner: UTA ||| Prediction: UTA WIN
2021-01-06: UTA @ NYK | winner: NYK ||| Prediction: UTA WIN
2021-01-10: UTA @ DET | winner: UTA ||| Prediction: UTA WIN
2021-05-26: MEM @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-04-17: UTA @ LAL | winner: LAL ||| Prediction: UTA WIN
2021-06-16: LAC @ UTA | winner: LAC ||| Prediction: UTA WIN
2020-12-31: PHO @ UTA | winner: PHO ||| Prediction: UTA WIN
2021-02-13: MIA @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-03-24: BRK @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-03-18: UTA @ WAS | winner: WAS ||| Prediction: UTA WIN
2021-02-22: CHO @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-01-31: UTA @ DEN | winner: DEN ||| Prediction: UTA WIN
2021-05-07: DEN @ UTA | winner: UTA ||| 

In [8]:
# Vanilla SVC
run(nba, model=SVC(random_state=0))

(84, 200, 6)
Team: Philadelphia 76ers accuracy = 52.38095238095239%
2021-03-03: UTA @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-02-15: PHI @ UTA | winner: UTA ||| Prediction: PHI WIN
2021-03-14: SAS @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-05-13: PHI @ MIA | winner: MIA ||| Prediction: PHI WIN
2021-04-21: PHO @ PHI | winner: PHO ||| Prediction: PHI WIN
2021-02-19: CHI @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-03-17: MIL @ PHI | winner: MIL ||| Prediction: PHI WIN
2021-06-20: ATL @ PHI | winner: ATL ||| Prediction: PHI WIN
2021-04-03: MIN @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-06-14: PHI @ ATL | winner: ATL ||| Prediction: PHI WIN
2021-06-08: ATL @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-02-06: BRK @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-02-04: POR @ PHI | winner: POR ||| Prediction: PHI WIN
2021-03-27: PHI @ LAC | winner: LAC ||| Prediction: PHI WIN
2021-05-31: PHI @ WAS | winner: WAS ||| Prediction: PHI WIN
2021-05-26: WAS @ PHI | winner: 

(79, 200, 6)
Team: Dallas Mavericks accuracy = 60.0%
2021-04-18: SAC @ DAL | winner: SAC ||| Prediction: DAL WIN
2021-03-03: OKC @ DAL | winner: DAL ||| Prediction: DAL WIN
2021-04-08: MIL @ DAL | winner: DAL ||| Prediction: DAL WIN
2021-05-02: SAC @ DAL | winner: SAC ||| Prediction: DAL WIN
2021-05-06: BRK @ DAL | winner: DAL ||| Prediction: DAL WIN
2021-01-30: PHO @ DAL | winner: PHO ||| Prediction: DAL WIN
2021-04-27: DAL @ GSW | winner: DAL ||| Prediction: DAL WIN
2021-02-14: POR @ DAL | winner: POR ||| Prediction: DAL WIN
2021-02-27: DAL @ BRK | winner: DAL ||| Prediction: DAL WIN
2021-03-24: DAL @ MIN | winner: DAL ||| Prediction: DAL WIN
2021-06-06: DAL @ LAC | winner: LAC ||| Prediction: DAL WIN
2021-02-25: DAL @ PHI | winner: PHI ||| Prediction: DAL WIN
2021-05-28: LAC @ DAL | winner: LAC ||| Prediction: DAL WIN
2021-03-17: LAC @ DAL | winner: DAL ||| Prediction: DAL WIN
2021-03-01: DAL @ ORL | winner: DAL ||| Prediction: DAL WIN
2021-03-31: DAL @ BOS | winner: DAL ||| Predict

(79, 200, 6)
Team: Los Angeles Lakers accuracy = 60.0%
2021-02-20: MIA @ LAL | winner: MIA ||| Prediction: LAL WIN
2020-12-30: LAL @ SAS | winner: LAL ||| Prediction: LAL WIN
2021-04-17: UTA @ LAL | winner: LAL ||| Prediction: LAL WIN
2021-03-25: PHI @ LAL | winner: PHI ||| Prediction: LAL WIN
2021-03-12: IND @ LAL | winner: LAL ||| Prediction: LAL WIN
2021-04-12: LAL @ NYK | winner: NYK ||| Prediction: LAL WIN
2021-01-18: GSW @ LAL | winner: GSW ||| Prediction: LAL WIN
2021-05-25: LAL @ PHO | winner: LAL ||| Prediction: LAL WIN
2021-03-21: LAL @ PHO | winner: PHO ||| Prediction: LAL WIN
2021-02-26: POR @ LAL | winner: LAL ||| Prediction: LAL WIN
2021-02-16: LAL @ MIN | winner: LAL ||| Prediction: LAL WIN
2021-04-08: LAL @ MIA | winner: MIA ||| Prediction: LAL WIN
2021-01-15: NOP @ LAL | winner: LAL ||| Prediction: LAL WIN
2020-12-27: MIN @ LAL | winner: LAL ||| Prediction: LAL WIN
2020-12-28: POR @ LAL | winner: POR ||| Prediction: LAL WIN
2021-04-24: LAL @ DAL | winner: DAL ||| Predi

(72, 200, 6)
Team: Oklahoma City Thunder accuracy = 72.22222222222221%
2021-05-02: PHO @ OKC | winner: PHO ||| Prediction: OKC LOSS
2021-02-17: OKC @ MEM | winner: MEM ||| Prediction: OKC LOSS
2021-05-16: LAC @ OKC | winner: OKC ||| Prediction: OKC LOSS
2021-03-24: MEM @ OKC | winner: MEM ||| Prediction: OKC LOSS
2021-01-02: OKC @ ORL | winner: OKC ||| Prediction: OKC LOSS
2021-04-19: OKC @ WAS | winner: WAS ||| Prediction: OKC LOSS
2021-01-10: OKC @ BRK | winner: OKC ||| Prediction: OKC LOSS
2021-02-10: OKC @ LAL | winner: LAL ||| Prediction: OKC LOSS
2021-03-18: OKC @ ATL | winner: ATL ||| Prediction: OKC LOSS
2021-02-01: HOU @ OKC | winner: HOU ||| Prediction: OKC LOSS
2021-02-12: OKC @ DEN | winner: DEN ||| Prediction: OKC LOSS
2021-03-31: TOR @ OKC | winner: OKC ||| Prediction: OKC LOSS
2021-03-03: OKC @ DAL | winner: DAL ||| Prediction: OKC LOSS
2021-04-03: OKC @ POR | winner: POR ||| Prediction: OKC LOSS
2021-04-18: OKC @ TOR | winner: TOR ||| Prediction: OKC LOSS
2021-02-06: MI

(83, 200, 6)
Team: Utah Jazz accuracy = 76.19047619047619%
2021-04-02: CHI @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-06-12: UTA @ LAC | winner: LAC ||| Prediction: UTA WIN
2021-02-27: UTA @ ORL | winner: UTA ||| Prediction: UTA WIN
2021-04-10: SAC @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-06-08: LAC @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-02-17: UTA @ LAC | winner: UTA ||| Prediction: UTA WIN
2021-03-27: MEM @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-04-24: MIN @ UTA | winner: MIN ||| Prediction: UTA WIN
2021-04-30: UTA @ PHO | winner: PHO ||| Prediction: UTA WIN
2021-01-27: DAL @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-05-26: MEM @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-01-06: UTA @ NYK | winner: NYK ||| Prediction: UTA WIN
2021-03-22: UTA @ CHI | winner: UTA ||| Prediction: UTA WIN
2021-04-16: IND @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-01-26: NYK @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-04-26: UTA @ MIN | winner: MIN ||| P

In [9]:
# defaults: C=1.0 gamma='scale', degree=3
kernels = ['rbf', 'poly']
gammas = [0.1, 1, 10, 100]
cs = [0.1, 1, 10, 100, 1000]
degrees = [0, 1, 2, 3, 4, 5, 6]

In [10]:
# Tuned SVC - rbf kernel
model = SVC(kernel=kernels[0], random_state=0)
run(nba, model=model)

(84, 200, 6)
Team: Philadelphia 76ers accuracy = 66.66666666666666%
2021-03-01: IND @ PHI | winner: PHI ||| Prediction: PHI WIN
2020-12-31: PHI @ ORL | winner: PHI ||| Prediction: PHI WIN
2021-04-03: MIN @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-04-09: PHI @ NOP | winner: NOP ||| Prediction: PHI WIN
2021-04-28: ATL @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-03-16: NYK @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-01-12: MIA @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-02-03: PHI @ CHO | winner: PHI ||| Prediction: PHI WIN
2021-02-21: PHI @ TOR | winner: TOR ||| Prediction: PHI WIN
2021-02-19: CHI @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-02-27: CLE @ PHI | winner: CLE ||| Prediction: PHI WIN
2021-06-06: ATL @ PHI | winner: ATL ||| Prediction: PHI WIN
2021-05-03: PHI @ CHI | winner: PHI ||| Prediction: PHI WIN
2021-01-22: BOS @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-01-31: PHI @ IND | winner: PHI ||| Prediction: PHI WIN
2021-04-24: PHI @ MIL | winner: 

(79, 200, 6)
Team: Dallas Mavericks accuracy = 65.0%
2021-01-27: DAL @ UTA | winner: UTA ||| Prediction: DAL WIN
2021-06-02: DAL @ LAC | winner: DAL ||| Prediction: DAL WIN
2021-03-17: LAC @ DAL | winner: DAL ||| Prediction: DAL WIN
2021-02-14: POR @ DAL | winner: POR ||| Prediction: DAL WIN
2021-04-18: SAC @ DAL | winner: SAC ||| Prediction: DAL WIN
2021-05-14: TOR @ DAL | winner: DAL ||| Prediction: DAL WIN
2021-01-23: HOU @ DAL | winner: HOU ||| Prediction: DAL WIN
2021-01-07: DAL @ DEN | winner: DAL ||| Prediction: DAL WIN
2021-02-27: DAL @ BRK | winner: DAL ||| Prediction: DAL WIN
2021-02-23: BOS @ DAL | winner: DAL ||| Prediction: DAL WIN
2021-02-12: NOP @ DAL | winner: DAL ||| Prediction: DAL WIN
2021-05-25: DAL @ LAC | winner: DAL ||| Prediction: DAL WIN
2021-03-27: DAL @ NOP | winner: NOP ||| Prediction: DAL WIN
2021-04-11: SAS @ DAL | winner: SAS ||| Prediction: DAL WIN
2021-01-29: DAL @ UTA | winner: UTA ||| Prediction: DAL WIN
2020-12-27: DAL @ LAC | winner: DAL ||| Predict

(79, 200, 6)
Team: Los Angeles Lakers accuracy = 60.0%
2021-02-14: LAL @ DEN | winner: DEN ||| Prediction: LAL WIN
2021-01-23: LAL @ CHI | winner: LAL ||| Prediction: LAL WIN
2021-03-16: MIN @ LAL | winner: LAL ||| Prediction: LAL WIN
2020-12-22: LAC @ LAL | winner: LAC ||| Prediction: LAL WIN
2021-01-30: LAL @ BOS | winner: LAL ||| Prediction: LAL WIN
2021-05-03: DEN @ LAL | winner: LAL ||| Prediction: LAL WIN
2021-02-12: MEM @ LAL | winner: LAL ||| Prediction: LAL WIN
2021-03-23: LAL @ NOP | winner: NOP ||| Prediction: LAL WIN
2021-02-24: LAL @ UTA | winner: UTA ||| Prediction: LAL WIN
2021-01-07: SAS @ LAL | winner: SAS ||| Prediction: LAL WIN
2021-01-15: NOP @ LAL | winner: LAL ||| Prediction: LAL WIN
2021-05-27: PHO @ LAL | winner: LAL ||| Prediction: LAL WIN
2021-02-08: OKC @ LAL | winner: LAL ||| Prediction: LAL WIN
2021-04-08: LAL @ MIA | winner: MIA ||| Prediction: LAL WIN
2021-03-12: IND @ LAL | winner: LAL ||| Prediction: LAL WIN
2021-03-28: ORL @ LAL | winner: LAL ||| Predi

(72, 200, 6)
Team: Oklahoma City Thunder accuracy = 61.111111111111114%
2021-04-07: CHO @ OKC | winner: CHO ||| Prediction: OKC LOSS
2021-05-14: UTA @ OKC | winner: UTA ||| Prediction: OKC LOSS
2021-03-31: TOR @ OKC | winner: OKC ||| Prediction: OKC LOSS
2021-03-11: DAL @ OKC | winner: OKC ||| Prediction: OKC LOSS
2021-03-29: DAL @ OKC | winner: DAL ||| Prediction: OKC LOSS
2021-01-19: OKC @ DEN | winner: DEN ||| Prediction: OKC LOSS
2021-01-12: SAS @ OKC | winner: SAS ||| Prediction: OKC LOSS
2020-12-31: NOP @ OKC | winner: NOP ||| Prediction: OKC LOSS
2021-02-12: OKC @ DEN | winner: DEN ||| Prediction: OKC LOSS
2021-02-05: MIN @ OKC | winner: MIN ||| Prediction: OKC LOSS
2021-02-06: MIN @ OKC | winner: OKC ||| Prediction: OKC LOSS
2021-04-26: OKC @ PHI | winner: PHI ||| Prediction: OKC LOSS
2021-05-16: LAC @ OKC | winner: OKC ||| Prediction: OKC LOSS
2021-01-22: OKC @ LAC | winner: LAC ||| Prediction: OKC LOSS
2021-01-10: OKC @ BRK | winner: OKC ||| Prediction: OKC LOSS
2021-02-03: H

(83, 200, 6)
Team: Utah Jazz accuracy = 76.19047619047619%
2021-02-13: MIA @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-03-31: UTA @ MEM | winner: UTA ||| Prediction: UTA WIN
2021-01-10: UTA @ DET | winner: UTA ||| Prediction: UTA WIN
2021-03-12: HOU @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-03-27: MEM @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-03-24: BRK @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-04-13: OKC @ UTA | winner: UTA ||| Prediction: UTA WIN
2020-12-23: UTA @ POR | winner: UTA ||| Prediction: UTA WIN
2021-06-18: UTA @ LAC | winner: LAC ||| Prediction: UTA WIN
2021-05-29: UTA @ MEM | winner: UTA ||| Prediction: UTA WIN
2021-04-03: ORL @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-03-26: MEM @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-04-07: UTA @ PHO | winner: PHO ||| Prediction: UTA WIN
2021-01-27: DAL @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-02-17: UTA @ LAC | winner: UTA ||| Prediction: UTA WIN
2021-01-06: UTA @ NYK | winner: NYK ||| P

In [11]:
# Tuned SVC - poly kernel
model = SVC(kernel=kernels[1], random_state=0)
run(nba, model=model)

(84, 200, 6)
Team: Philadelphia 76ers accuracy = 61.904761904761905%
2021-03-03: UTA @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-06-06: ATL @ PHI | winner: ATL ||| Prediction: PHI WIN
2020-12-26: PHI @ NYK | winner: PHI ||| Prediction: PHI WIN
2021-01-14: MIA @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-01-06: WAS @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-02-06: BRK @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-03-23: PHI @ GSW | winner: PHI ||| Prediction: PHI WIN
2021-01-20: BOS @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-01-27: LAL @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-01-16: PHI @ MEM | winner: MEM ||| Prediction: PHI WIN
2021-01-07: PHI @ BRK | winner: BRK ||| Prediction: PHI WIN
2021-02-11: PHI @ POR | winner: POR ||| Prediction: PHI WIN
2021-06-02: WAS @ PHI | winner: PHI ||| Prediction: PHI WIN
2021-03-25: PHI @ LAL | winner: PHI ||| Prediction: PHI WIN
2021-04-22: PHI @ MIL | winner: MIL ||| Prediction: PHI WIN
2021-03-30: PHI @ DEN | winner:

(79, 200, 6)
Team: Dallas Mavericks accuracy = 50.0%
2021-05-04: DAL @ MIA | winner: DAL ||| Prediction: DAL WIN
2021-04-03: DAL @ WAS | winner: DAL ||| Prediction: DAL WIN
2021-01-29: DAL @ UTA | winner: UTA ||| Prediction: DAL LOSS
2021-03-21: DAL @ POR | winner: DAL ||| Prediction: DAL LOSS
2021-05-11: DAL @ MEM | winner: MEM ||| Prediction: DAL LOSS
2021-02-12: NOP @ DAL | winner: DAL ||| Prediction: DAL LOSS
2021-05-06: BRK @ DAL | winner: DAL ||| Prediction: DAL LOSS
2021-04-16: NYK @ DAL | winner: NYK ||| Prediction: DAL LOSS
2021-04-27: DAL @ GSW | winner: DAL ||| Prediction: DAL WIN
2021-04-08: MIL @ DAL | winner: DAL ||| Prediction: DAL LOSS
2021-02-08: MIN @ DAL | winner: DAL ||| Prediction: DAL LOSS
2021-05-01: WAS @ DAL | winner: DAL ||| Prediction: DAL LOSS
2021-04-29: DAL @ DET | winner: DAL ||| Prediction: DAL LOSS
2021-05-25: DAL @ LAC | winner: DAL ||| Prediction: DAL LOSS
2021-05-14: TOR @ DAL | winner: DAL ||| Prediction: DAL LOSS
2021-02-04: GSW @ DAL | winner: GSW

(79, 200, 6)
Team: Los Angeles Lakers accuracy = 50.0%
2021-04-02: LAL @ SAC | winner: LAL ||| Prediction: LAL WIN
2021-01-10: LAL @ HOU | winner: LAL ||| Prediction: LAL WIN
2021-02-28: GSW @ LAL | winner: LAL ||| Prediction: LAL WIN
2021-04-26: LAL @ ORL | winner: LAL ||| Prediction: LAL LOSS
2021-01-08: CHI @ LAL | winner: LAL ||| Prediction: LAL LOSS
2020-12-28: POR @ LAL | winner: POR ||| Prediction: LAL WIN
2021-05-11: NYK @ LAL | winner: LAL ||| Prediction: LAL WIN
2021-04-28: LAL @ WAS | winner: WAS ||| Prediction: LAL WIN
2021-04-19: UTA @ LAL | winner: UTA ||| Prediction: LAL WIN
2021-04-30: SAC @ LAL | winner: SAC ||| Prediction: LAL WIN
2021-01-21: LAL @ MIL | winner: LAL ||| Prediction: LAL WIN
2021-01-13: LAL @ OKC | winner: LAL ||| Prediction: LAL WIN
2021-02-18: BRK @ LAL | winner: BRK ||| Prediction: LAL WIN
2021-04-04: LAL @ LAC | winner: LAC ||| Prediction: LAL WIN
2021-05-07: LAL @ POR | winner: POR ||| Prediction: LAL LOSS
2021-01-18: GSW @ LAL | winner: GSW ||| Pr

(72, 200, 6)
Team: Oklahoma City Thunder accuracy = 72.22222222222221%
2021-03-14: MEM @ OKC | winner: OKC ||| Prediction: OKC LOSS
2021-03-11: DAL @ OKC | winner: OKC ||| Prediction: OKC LOSS
2021-04-29: NOP @ OKC | winner: NOP ||| Prediction: OKC LOSS
2021-02-19: OKC @ MIL | winner: MIL ||| Prediction: OKC LOSS
2021-04-03: OKC @ POR | winner: POR ||| Prediction: OKC LOSS
2021-03-31: TOR @ OKC | winner: OKC ||| Prediction: OKC LOSS
2021-03-03: OKC @ DAL | winner: DAL ||| Prediction: OKC LOSS
2021-03-29: DAL @ OKC | winner: DAL ||| Prediction: OKC LOSS
2021-02-08: OKC @ LAL | winner: LAL ||| Prediction: OKC LOSS
2021-02-06: MIN @ OKC | winner: OKC ||| Prediction: OKC LOSS
2021-05-04: SAC @ OKC | winner: SAC ||| Prediction: OKC LOSS
2021-02-05: MIN @ OKC | winner: MIN ||| Prediction: OKC LOSS
2021-01-08: OKC @ NYK | winner: OKC ||| Prediction: OKC LOSS
2021-04-26: OKC @ PHI | winner: PHI ||| Prediction: OKC LOSS
2021-01-12: SAS @ OKC | winner: SAS ||| Prediction: OKC LOSS
2021-02-22: MI

(83, 200, 6)
Team: Utah Jazz accuracy = 61.904761904761905%
2021-03-03: UTA @ PHI | winner: PHI ||| Prediction: UTA WIN
2021-01-17: UTA @ DEN | winner: UTA ||| Prediction: UTA WIN
2021-02-26: UTA @ MIA | winner: MIA ||| Prediction: UTA WIN
2021-05-01: TOR @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-04-19: UTA @ LAL | winner: UTA ||| Prediction: UTA WIN
2021-04-28: UTA @ SAC | winner: UTA ||| Prediction: UTA WIN
2021-02-27: UTA @ ORL | winner: UTA ||| Prediction: UTA WIN
2021-04-12: WAS @ UTA | winner: WAS ||| Prediction: UTA WIN
2021-01-23: GSW @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-04-07: UTA @ PHO | winner: PHO ||| Prediction: UTA WIN
2021-06-10: LAC @ UTA | winner: UTA ||| Prediction: UTA WIN
2021-03-31: UTA @ MEM | winner: UTA ||| Prediction: UTA WIN
2021-04-17: UTA @ LAL | winner: LAL ||| Prediction: UTA WIN
2021-01-12: UTA @ CLE | winner: UTA ||| Prediction: UTA WIN
2021-03-14: UTA @ GSW | winner: GSW ||| Prediction: UTA WIN
2021-01-31: UTA @ DEN | winner: DEN ||| 

In [ ]:
%%time

# Tuned SVC - rbf kernel + gammas
for gamma in gammas:
    print(f'gamma = {gamma}')
    model = SVC(kernel=kernels[0], random_state=0, gamma=gamma)
    run(nba, model=model, printouts=False)

In [ ]:
%%time

# Tuned SVC - poly kernel
for gamma in gammas:
    print(f'gamma = {gamma}')
    model = SVC(kernel=kernels[1], random_state=0, gamma=gamma)
    run(nba, model=model, printouts=False)

In [ ]:
%%time

# Tuned SVC - rbf kernel + C
for c in cs:
    print(f'c = {c}')
    model = SVC(kernel=kernels[0], random_state=0, C=c)
    run(nba, model=model, printouts=False)

In [ ]:
%%time

# Tuned SVC - poly kernel + C
for c in cs:
    print(f'c = {c}')
    model = SVC(kernel=kernels[1], random_state=0, C=c)
    run(nba, model=model, printouts=False)